## Terrain Classification using Convolutional Neural Networks
This notebook demonstrates how to build and train a Convolutional Neural Network (CNN) for classifying terrain images into five categories.

### Installing necessary libraries

In [ ]:
pip install tensorflow
pip install scipy

## Importing Libraries

Now we'll import the necessary libraries and modules.

In [62]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau


### Model Architecture

We'll define a CNN model for our terrain classification task.

In [71]:
model = tf.keras.Sequential([

    # Input layer
    tf.keras.layers.Input(shape=(64, 64, 3)),
    
    # First convolutional layer
    tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Second convolutional layer
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Fully connected classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])


### Model Compilation

We'll compile the model with appropriate loss function and optimizer.

In [72]:
# Could also use sparse_categorical_crossentropy if they are indices instead of hot encoded labels
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Data Preprocessing

We'll set up data generators for training and testing data, including data augmentation for the training set.

In [73]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=30,
                                   brightness_range=[0.8, 1.2],
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [74]:
# Update the directory paths to point to your local Downloads folder
training_set = train_datagen.flow_from_directory(r'C:\Users\draka\Downloads\Terrain_Images\Training Data',
                                                 target_size=(64, 64),
                                                 batch_size=64,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(r'C:\Users\draka\Downloads\Terrain_Images\Testing Data',
                                            target_size=(64, 64),
                                            batch_size=64,
                                            class_mode='categorical')


Found 10000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


### Learning Rate Scheduling

We'll define a learning rate schedule to adjust the learning rate during training.

In [75]:
# Define the learning rate schedule
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-2
    else:
        return 1e-3

lr_schedule = LearningRateScheduler(scheduler)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


### Model Training

Now we'll train the model on our dataset.

In [77]:
modelZ = model.fit(
    training_set,
    steps_per_epoch=5000,
    epochs=10,
    validation_data=test_set,
    validation_steps=7
)

Epoch 1/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.6969 - loss: 0.7870 - val_accuracy: 0.7321 - val_loss: 0.7718
Epoch 2/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.7108 - loss: 0.7600 - val_accuracy: 0.7692 - val_loss: 0.7620
Epoch 3/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7157 - loss: 0.7431 - val_accuracy: 0.7433 - val_loss: 0.6570
Epoch 4/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.7263 - loss: 0.7289 - val_accuracy: 0.7885 - val_loss: 0.6065
Epoch 5/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7320 - loss: 0.7129 - val_accuracy: 0.7812 - val_loss: 0.5878
Epoch 6/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.7377 - loss: 0.6902 - val_accuracy: 0.8269 - val_loss: 0.4793
Epoch 7/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 36s 7ms/step - accuracy: 0.7383 - loss: 0.6886 - val_accuracy: 0.8058 - val_loss: 0.5368
Epoch 8/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.7497 - loss: 0

### GPU Availability Check

Let's check if any GPUs are available for training acceleration.

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
